In [1]:
import requests
import pandas as pd
import mysql.connector
import time
import ast
import warnings

In [2]:
database_name = 'league-database'
username = 'admin'
password = 'CowDog019283'
endpoint = 'league-database.cgdkdqtwgxcv.us-east-2.rds.amazonaws.com'
port = 3306
db_name = 'leaguedb'

In [3]:
warnings.simplefilter(action='ignore', category=UserWarning)

In [5]:
connection = mysql.connector.connect(host=endpoint, database='leaguedb', user=username, password=password)
cursor = connection.cursor()
query = "SELECT * FROM matchIDs LIMIT 5"
cursor.execute(query)

In [6]:
data = cursor.fetchall()

In [42]:
query = '''ALTER TABLE games
        DROP COLUMN champion,
        DROP COLUMN lane,
        DROP COLUMN spell1,
        DROP COLUMN spell2,
        DROP COLUMN kills,
        DROP COLUMN deaths,
        DROP COLUMN assists,
        DROP COLUMN totalDamageDealt,
        DROP COLUMN goldEarned,
        DROP COLUMN champLevel,
        DROP COLUMN csPermin,
        DROP COLUMN item0,
        DROP COLUMN item1,
        DROP COLUMN item2,
        DROP COLUMN item3,
        DROP COLUMN item4,
        DROP COLUMN item5,
        DROP COLUMN item6,
        DROP COLUMN runes;'''
cursor.execute(query)


In [49]:
query = '''ALTER TABLE games
        ADD summoner1 TEXT,
        ADD summoner2 TEXT,
        ADD summoner3 TEXT,
        ADD summoner4 TEXT,
        ADD summoner5 TEXT,
        ADD summoner6 TEXT,
        ADD summoner7 TEXT,
        ADD summoner8 TEXT,
        ADD summoner9 TEXT,
        ADD summoner10 TEXT;'''
cursor.execute(query)

In [55]:
query = '''ALTER TABLE games
        ADD duration TINYTEXT,
        ADD gameid VARCHAR(30)'''
cursor.execute(query)

In [73]:
query = '''ALTER TABLE games
        ADD allSummoners TEXT;'''
cursor.execute(query)

In [10]:
query = '''ALTER TABLE games
        ADD gamemode TINYTEXT'''
cursor.execute(query)

In [50]:
query = '''ALTER TABLE games
        DROP COLUMN gameid,
        DROP COLUMN summoner,
        DROP COLUMN gamerank,
        DROP COLUMN win,
        DROP COLUMN time;'''
cursor.execute(query)

In [11]:
query = "DESCRIBE games"
cursor.execute(query)
cursor.fetchall()

[('summoner1', b'text', 'YES', bytearray(b''), None, ''),
 ('summoner2', b'text', 'YES', bytearray(b''), None, ''),
 ('summoner3', b'text', 'YES', bytearray(b''), None, ''),
 ('summoner4', b'text', 'YES', bytearray(b''), None, ''),
 ('summoner5', b'text', 'YES', bytearray(b''), None, ''),
 ('summoner6', b'text', 'YES', bytearray(b''), None, ''),
 ('summoner7', b'text', 'YES', bytearray(b''), None, ''),
 ('summoner8', b'text', 'YES', bytearray(b''), None, ''),
 ('summoner9', b'text', 'YES', bytearray(b''), None, ''),
 ('summoner10', b'text', 'YES', bytearray(b''), None, ''),
 ('duration', b'tinytext', 'YES', bytearray(b''), None, ''),
 ('gameid', b'varchar(30)', 'YES', bytearray(b''), None, ''),
 ('allSummoners', b'text', 'YES', bytearray(b''), None, ''),
 ('gamemode', b'tinytext', 'YES', bytearray(b''), None, '')]

In [6]:
def duration(minutes):
    mins = minutes / 60
    spt = str(mins).split('.')
    m = int(spt[0])
    seconds = round(float('.' + spt[1]) * 60)
    # To prevent dividing by 0
    if mins == 0:
        mins = 1
    return [str(m) +'m ' + str(seconds) + 's', mins]

In [104]:
def game_info(data, counter):
    try:
        if counter >= len(data):
            return None
        for info in data[counter:]:
            
            participants = []
            lit = ast.literal_eval(info)
            dur = lit['info']['gameDuration']
            mins = duration(dur)
            game_id = lit['metadata']['matchId']
            game_mode = lit['info']['gameMode']
            add_summoner = (
            '''INSERT INTO games(
                summoner1, summoner2, summoner3, summoner4, summoner5, summoner6, summoner7, summoner8, summoner9, summoner10, duration, gameid, allSummoners, gamemode)
                VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''
        )
            for row in lit['info']['participants']:
                participants_row = {}
                participants_row['summoner'] = row['summonerName']
                participants_row['champion'] = row['championId']
                participants_row['spell1'] = row['summoner1Id']
                participants_row['spell2'] = row['summoner2Id']
                participants_row['kills'] = row['kills']
                participants_row['deaths'] = row['deaths']
                participants_row['assists'] = row['assists']
                participants_row['totalDamageDealt'] = row['totalDamageDealt']
                participants_row['goldEarned'] = row['goldEarned']
                participants_row['champLevel'] = row['champLevel']
                participants_row['csPerMin'] = round(row['totalMinionsKilled'] / mins[1], 1)
                participants_row['totalMinionsKilled'] = row['totalMinionsKilled']
                participants_row['item1'] = row['item0']
                participants_row['item2'] = row['item1']
                participants_row['item3'] = row['item2']
                participants_row['item4'] = row['item3']
                participants_row['item5'] = row['item4']
                participants_row['item6'] = row['item5']
                participants_row['item7'] = row['item6']
                participants_row['rune1'] = row['perks']['styles'][0]['selections'][0]['perk']
                participants_row['rune2'] = row['perks']['styles'][0]['selections'][1]['perk']
                participants_row['rune3'] = row['perks']['styles'][0]['selections'][2]['perk']
                participants_row['rune4'] = row['perks']['styles'][0]['selections'][3]['perk']
                participants_row['rune5'] = row['perks']['styles'][1]['selections'][0]['perk']
                participants_row['rune6'] = row['perks']['styles'][1]['selections'][1]['perk']
                participants_row['win'] = row['win']
                participants.append(str(participants_row))
            summoners = (participants[0], participants[1], participants[2], participants[3], participants[4], participants[5], participants[6], participants[7], participants[8], participants[9], mins[0], game_id , str(participants), game_mode)

            cursor.execute(add_summoner, summoners)
            connection.commit()

            counter += 1
            if counter == len(data):
                return None

    except IndexError:
        counter += 1
        print('index problem, keep going', counter)
        if counter < len(data):
            game_info(data, counter)

        else:
            return None
    else: 
        raise

In [9]:
one = pd.read_csv('one.csv')
two = pd.read_csv('two.csv')
three = pd.read_csv('three.csv')
four = pd.read_csv('four.csv')
five = pd.read_csv('five.csv')
six = pd.read_csv('six.csv')
seven = pd.read_csv('seven.csv')
rest = pd.read_csv('last.csv')

In [84]:
import re
test = str(one['info'][737])
test1 = test.replace('\\', '')
test1

'{\'metadata\': {\'dataVersion\': \'2\', \'matchId\': \'NA1_4289491925\', \'participants\': [\'uOGoiETHskOV4NiaFKpmV9Mtcux9DJPrs-w-9M5ZOpIYI5q_ojlJrlAEnifTTr1RSIxWgbrtHDu4gw\']}, \'info\': {\'gameCreation\': 1651042200000, \'gameDuration\': 93, \'gameEndTimestamp\': 1651042362050, \'gameId\': 4289491925, \'gameMode\': \'PRACTICETOOL\', \'gameName\': "23eft\'s game", \'gameStartTimestamp\': 1651042268889, \'gameType\': \'CUSTOM_GAME\', \'gameVersion\': \'12.7.433.4138\', \'mapId\': 11, \'participants\': [{\'assists\': 0, \'baronKills\': 0, \'bountyLevel\': 0, \'champExperience\': 0, \'champLevel\': 1, \'championId\': 131, \'championName\': \'Diana\', \'championTransform\': 0, \'consumablesPurchased\': 0, \'damageDealtToBuildings\': 0, \'damageDealtToObjectives\': 0, \'damageDealtToTurrets\': 0, \'damageSelfMitigated\': 0, \'deaths\': 0, \'detectorWardsPlaced\': 0, \'doubleKills\': 0, \'dragonKills\': 0, \'eligibleForProgression\': True, \'firstBloodAssist\': False, \'firstBloodKill\': F

In [83]:
test2 = re.sub(r'\\', '', test)
test2

'{\'metadata\': {\'dataVersion\': \'2\', \'matchId\': \'NA1_4289491925\', \'participants\': [\'uOGoiETHskOV4NiaFKpmV9Mtcux9DJPrs-w-9M5ZOpIYI5q_ojlJrlAEnifTTr1RSIxWgbrtHDu4gw\']}, \'info\': {\'gameCreation\': 1651042200000, \'gameDuration\': 93, \'gameEndTimestamp\': 1651042362050, \'gameId\': 4289491925, \'gameMode\': \'PRACTICETOOL\', \'gameName\': "23eft\'s game", \'gameStartTimestamp\': 1651042268889, \'gameType\': \'CUSTOM_GAME\', \'gameVersion\': \'12.7.433.4138\', \'mapId\': 11, \'participants\': [{\'assists\': 0, \'baronKills\': 0, \'bountyLevel\': 0, \'champExperience\': 0, \'champLevel\': 1, \'championId\': 131, \'championName\': \'Diana\', \'championTransform\': 0, \'consumablesPurchased\': 0, \'damageDealtToBuildings\': 0, \'damageDealtToObjectives\': 0, \'damageDealtToTurrets\': 0, \'damageSelfMitigated\': 0, \'deaths\': 0, \'detectorWardsPlaced\': 0, \'doubleKills\': 0, \'dragonKills\': 0, \'eligibleForProgression\': True, \'firstBloodAssist\': False, \'firstBloodKill\': F

In [82]:
# Since none of thest work, we will just be ignoring them
test3 = ''
for t in test1:
    test3 += t
test3

'{\'metadata\': {\'dataVersion\': \'2\', \'matchId\': \'NA1_4289491925\', \'participants\': [\'uOGoiETHskOV4NiaFKpmV9Mtcux9DJPrs-w-9M5ZOpIYI5q_ojlJrlAEnifTTr1RSIxWgbrtHDu4gw\']}, \'info\': {\'gameCreation\': 1651042200000, \'gameDuration\': 93, \'gameEndTimestamp\': 1651042362050, \'gameId\': 4289491925, \'gameMode\': \'PRACTICETOOL\', \'gameName\': "23eft\'s game", \'gameStartTimestamp\': 1651042268889, \'gameType\': \'CUSTOM_GAME\', \'gameVersion\': \'12.7.433.4138\', \'mapId\': 11, \'participants\': [{\'assists\': 0, \'baronKills\': 0, \'bountyLevel\': 0, \'champExperience\': 0, \'champLevel\': 1, \'championId\': 131, \'championName\': \'Diana\', \'championTransform\': 0, \'consumablesPurchased\': 0, \'damageDealtToBuildings\': 0, \'damageDealtToObjectives\': 0, \'damageDealtToTurrets\': 0, \'damageSelfMitigated\': 0, \'deaths\': 0, \'detectorWardsPlaced\': 0, \'doubleKills\': 0, \'dragonKills\': 0, \'eligibleForProgression\': True, \'firstBloodAssist\': False, \'firstBloodKill\': F

In [114]:
files = [one, two, three, four, five, six, seven, rest]
count = 0
for file in files:
    count += 1
    print('file number', count)
    game_info(file['info'], 0)

file number 1
index problem, keep going 738
index problem, keep going 791
index problem, keep going 1143
index problem, keep going 1657
index problem, keep going 1658
index problem, keep going 1659
index problem, keep going 1697
index problem, keep going 1763
index problem, keep going 2118
index problem, keep going 2669
index problem, keep going 2691
index problem, keep going 2694
index problem, keep going 2993
index problem, keep going 2994
index problem, keep going 2995
index problem, keep going 3017
index problem, keep going 3018
index problem, keep going 3019
index problem, keep going 3065
index problem, keep going 3405
index problem, keep going 3410
index problem, keep going 3454
index problem, keep going 3455
index problem, keep going 3456
index problem, keep going 3457
index problem, keep going 3458
index problem, keep going 3460
index problem, keep going 3851
index problem, keep going 3853
index problem, keep going 3854
index problem, keep going 3855
index problem, keep going 3

In [7]:
# This is after the first 10000, lost 61 rows
# After the second 10000 we lost, 24 rows
# After the third 10000, we lost 11 rows
# After the fourth 10000, we lost  26 rows
# After the fifth 10000, we lost 20 rows
# After the sixth 10000, we lost 3 rows
# After the seventh 10000, we lost 5 rows
# After the last 6407, we lost 16 rows
# Total loss of 161
query = "SELECT * FROM games"
pd.read_sql(query, connection)

,summoner1,summoner2,summoner3,summoner4,summoner5,summoner6,summoner7,summoner8,summoner9,summoner10,duration,gameid,allSummoners,gamemode
0,"{'summoner': 'KIMMYBO123', 'champion': 350, 's...","{'summoner': 'ishaw1', 'champion': 31, 'spell1...","{'summoner': 'JJ41411', 'champion': 67, 'spell...","{'summoner': 'Shoogz12', 'champion': 110, 'spe...","{'summoner': 'PixelGaMERCaT', 'champion': 203,...","{'summoner': 'zackvi', 'champion': 246, 'spell...","{'summoner': 'Soundwave2017', 'champion': 30, ...","{'summoner': 'ashtonXX', 'champion': 21, 'spel...","{'summoner': 'Badassjones', 'champion': 254, '...","{'summoner': 'psychoquack', 'champion': 89, 's...",20m 35s,NA1_4366795660,"[""{'summoner': 'KIMMYBO123', 'champion': 350, ...",ARAM
1,"{'summoner': 'Mina Sana Momo ', 'champion': 21...","{'summoner': 'McDflurry', 'champion': 41, 'spe...","{'summoner': 'boileddove69', 'champion': 67, '...","{'summoner': 'HymenPulverizr', 'champion': 53,...","{'summoner': 'Pattse', 'champion': 143, 'spell...","{'summoner': 'I Win at 20', 'champion': 42, 's...","{'summoner': 'FreeToKillU', 'champion': 133, '...","{'summoner': 'Soundwave2017', 'champion': 777,...","{'summoner': 'irishcb187', 'champion': 498, 's...","{'summoner': 'idontplaythemeta', 'champion': 3...",23m 55s,NA1_4366527467,"[""{'summoner': 'Mina Sana Momo ', 'champion': ...",ARAM
2,"{'summoner': 'TooMuchRice', 'champion': 89, 's...","{'summoner': 'OstrichJacket', 'champion': 41, ...","{'summoner': 'Soundwave2017', 'champion': 35, ...","{'summoner': 'MikuNakanoooo', 'champion': 67, ...","{'summoner': 'TenorWill', 'champion': 518, 'sp...","{'summoner': 'palalaz', 'champion': 34, 'spell...","{'summoner': 'oOholymolyOo', 'champion': 10, '...","{'summoner': 'JDDan', 'champion': 234, 'spell1...","{'summoner': 'Odobee', 'champion': 876, 'spell...","{'summoner': 'TenLog', 'champion': 107, 'spell...",20m 6s,NA1_4366533737,"[""{'summoner': 'TooMuchRice', 'champion': 89, ...",ARAM
3,"{'summoner': 'Karishalla', 'champion': 43, 'sp...","{'summoner': 'MrChubbin', 'champion': 150, 'sp...","{'summoner': 'ilikechips8', 'champion': 74, 's...","{'summoner': 'buckaroo22', 'champion': 234, 's...","{'summoner': 'LitCroissant', 'champion': 142, ...","{'summoner': 'Jason1991sens', 'champion': 235,...","{'summoner': 'MaInGn', 'champion': 104, 'spell...","{'summoner': 'GelatinousN00b', 'champion': 50,...","{'summoner': 'AVNOIR', 'champion': 134, 'spell...","{'summoner': 'Soundwave2017', 'champion': 61, ...",23m 53s,NA1_4365266709,"[""{'summoner': 'Karishalla', 'champion': 43, '...",ARAM
4,"{'summoner': 'Lemineraz', 'champion': 103, 'sp...","{'summoner': 'zzZaxiade', 'champion': 75, 'spe...","{'summoner': 'KnightMm', 'champion': 13, 'spel...","{'summoner': 'Soundwave2017', 'champion': 32, ...","{'summoner': 'Castles', 'champion': 122, 'spel...","{'summoner': 'bearsforlife54', 'champion': 96,...","{'summoner': 'SomQuite', 'champion': 266, 'spe...","{'summoner': 'jjjjjji95', 'champion': 17, 'spe...","{'summoner': 'Umbrellak08', 'champion': 99, 's...","{'summoner': 'hhh0000', 'champion': 117, 'spel...",15m 10s,NA1_4365293282,"[""{'summoner': 'Lemineraz', 'champion': 103, '...",ARAM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76241,"{'summoner': 'Dragoon', 'champion': 82, 'spell...","{'summoner': 'cant offrole', 'champion': 77, '...","{'summoner': 'Laceration', 'champion': 91, 'sp...","{'summoner': 'callum koi', 'champion': 360, 's...","{'summoner': 'Downy Hyung', 'champion': 888, '...","{'summoner': 'Meslo', 'champion': 41, 'spell1'...","{'summoner': 'wow player 123', 'champion': 163...","{'summoner': '0 n a t ', 'champion': 45, 'spel...","{'summoner': 'Previous1', 'champion': 119, 'sp...","{'summoner': 'WAK TOO', 'champion': 50, 'spell...",3m 13s,NA1_4363734624,"[""{'summoner': 'Dragoon', 'champion': 82, 'spe...",CLASSIC
76242,"{'summoner': 'jjking9', 'champion': 164, 'spel...","{'summoner': 'cant offrole', 'champion': 163, ...","{'summoner': 'Nim Chimpsky', 'champion': 3, 's...","{'summoner': '

In [8]:
query = '''ALTER TABLE games RENAME TO gameSummonerInfo'''
cursor.execute(query)

In [9]:
query = 'SHOW TABLES;'
cursor.execute(query)
cursor.fetchall()

[(bytearray(b'champions'),),
 (bytearray(b'gameSummonerInfo'),),
 (bytearray(b'items'),),
 (bytearray(b'matchIDs'),),
 (bytearray(b'runes'),),
 (bytearray(b'spells'),),
 (bytearray(b'summoners'),)]

In [8]:
cursor.close()
connection.close()